In [1]:
import matplotlib.pyplot as plt

import numpy as np

import cv2
import mtcnn

import os, sys

from tensorflow.keras import utils
from sklearn.utils import shuffle

In [2]:
face_detector = mtcnn.MTCNN()
def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)

In [3]:
# data location
data_dir = "F:/ai/recognition-football-players/data/train_faces"
os.chdir(data_dir)

# labels and images lists
train_labels = []
train_images = []

# players name
players = os.listdir()

# images count
images_count = 0

# class images count
class_image = -1

class_name = []

for player in players:
    class_image += 1
    
    class_name.append(player)

    # change directory
    os.chdir(f'{data_dir}/{player}')

    # all images for this player
    images_= os.listdir()

    for img_ in images_:
        # split image number and name
        num = img_.split('_')[0]
        name = player

        # load image
        image_ = cv2.imread(img_)
        img_rgb = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)

        # detect face
        image_ = face_detector.detect_faces(img_rgb)
        if image_:
            res = max(image_, key=lambda b: b['box'][2] * b['box'][3])
            face, _, _ = get_face(img_rgb, res['box'])
            face = cv2.resize(face, (224, 224))

        # add images array to images list
        train_images.append(face)
    
        # images_count = images_count + 1
        images_count += 1
        
        train_labels.append([class_image])

# images count
print(f'images count: {images_count}')
print(class_name)

images count: 420
['Ahmad-Nourollahi', 'Ali-Gholizadeh', 'Alireza-Beiranvand', 'Alireza-Jahanbakhsh', 'Ehsan-Hajsafi', 'Hamed-Lak', 'Hossein-Hosseini', 'Issa-Alekasir', 'Kamal-Kamyabinia', 'Karim-Ansarifard', 'Kaveh-Rezaei', 'Mehdi-Abdi', 'Mehdi-Shiri', 'Mehdi-Taremi', 'Mehdi-Torabi', 'Milad-Mohammadi', 'Milad-Sarlak', 'Mohammad-Hossein-Kanani-Zadegan', 'Omid-Alishah', 'Omid-Noorafkan', 'Sadegh-Moharrami', 'Saeed-Aghaei', 'Saeed-Ezatolahi', 'Saman-Ghoddos', 'Sardar-Azmoun', 'Seyed-Jalal-Hosseini', 'Shoja-Khalilzadeh', 'Vahid-Amiri']


In [4]:
# labels and images lists
test_labels = []
test_images = []

data_dir = "F:/ai/recognition-football-players/data/test_faces"
os.chdir(data_dir)

# players name
players = os.listdir()

# images count
images_count = 0

# class images count
class_image = -1

class_name = []

columns = 20
rows = 28
# fig = plt.figure(figsize=(120, 110))
for player in players:
    class_image += 1
    
    class_name.append(player)

    # change directory
    os.chdir(sys.path[0])
    os.chdir(r"F:\ai\recognition-football-players\data\test_faces")
    os.chdir(f'./{player}')
    
    # all images for this player
    images_= os.listdir()
    for img_ in images_:
        
        # split image number and name
        num = img_.split('_')[0]
        name = img_.split('_')[1][:-4]
            
        # load image
        image_ = cv2.imread(img_)
        img_rgb = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)

        # detect face
        image_ = face_detector.detect_faces(img_rgb)
        if image_:
            res = max(image_, key=lambda b: b['box'][2] * b['box'][3])
            face, _, _ = get_face(img_rgb, res['box'])
            face = cv2.resize(face, (224, 224))

        # add images array to images list
        test_images.append(face)
    
        # images_count = images_count + 1
        images_count += 1

        test_labels.append([class_image])

In [5]:
print(f"test images: {len(test_images)}, train images: {len(train_images)}, names: {len(class_name)}")

test images: 140, train images: 420, names: 28


In [6]:
train_labels = np.array(train_labels)
train_images = np.array(train_images).astype('float32')/255

test_labels = np.array(test_labels)
test_images = np.array(test_images).astype('float32')/255

test_labels = utils.to_categorical(test_labels)
train_labels = utils.to_categorical(train_labels)

train_images, train_labels = shuffle(train_images, train_labels)
test_images, test_labels = shuffle(test_images, test_labels)

print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)
np.savez_compressed(r"F:\ai\recognition-football-players\dataset", train_images=train_images, test_images=test_images, train_labels=train_labels, test_labels=test_labels, names=class_name)

(420, 224, 224, 3) (140, 224, 224, 3) (420, 28) (140, 28)
